# Egobox

## Installation

You have to get the right wheel archive regarding your Python/Platform installation.

Then you can install with the pip command as follows: (here for Python 3.8 on Windows)

pip install egobox-0.1.0-cp38-none-win_amd64.whl

## Import

In [11]:
import numpy as np
import egobox as egx

# To display optimization information (none by default)
import logging
logging.basicConfig(level=logging.INFO)

## Test functions 

In [12]:
xlimits_xsinx = np.array([[0., 25.]])
n_cstr_xsinx = 0

def xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    return y

In [13]:
xlimits_g24 = np.array([[0., 3.], [0., 4.]])
n_cstr_g24 = 2

# Objective
def G24(point):
    """
    Function g24
    1 global optimum y_opt = -5.5080 at x_opt =(2.3295, 3.1785)
    """
    p = np.atleast_2d(point)
    return - p[:, 0] - p[:, 1]

# Constraints < 0
def G24_c1(point):
    p = np.atleast_2d(point)
    return (- 2.0 * p[:, 0] ** 4.0
            + 8.0 * p[:, 0] ** 3.0 
            - 8.0 * p[:, 0] ** 2.0 
            + p[:, 1] - 2.0)

def G24_c2(point):
    p = np.atleast_2d(point)
    return (-4.0 * p[:, 0] ** 4.0
            + 32.0 * p[:, 0] ** 3.0
            - 88.0 * p[:, 0] ** 2.0
            + 96.0 * p[:, 0]
            + p[:, 1] - 36.0)

# Grouped evaluation
def g24(point):
    p = np.atleast_2d(point)
    return np.array([G24(p), G24_c1(p), G24_c2(p)]).T


## Egobox Rust optimizer (a.k.a Egor)

In [14]:
egor = egx.Optimizer(xlimits_g24, infill_strategy=egx.InfillStrategy.EI)  # see help(SegoOptimizer) for options

# Restrict regression and correlation models used
# egor = egx.Optimizer(xlimits_g24, infill_strategy=egx.InfillStrategy.EI,
#                      regr_spec=egx.RegressionSpec.LINEAR,
#                      corr_spec=egx.CorrelationSpec.MATERN32 | egx.CorrelationSpec.MATERN52)  

In [15]:
res = egor.minimize(g24, n_cstr=n_cstr_g24, n_eval=20)
print(f"Optimization f={res.y_opt} at {res.x_opt}")

INFO:moe.algorithm:Best expert Linear_SquaredExponential accuracy=0.000000000000000044408920985006264
INFO:moe.algorithm:Best expert Linear_SquaredExponential accuracy=0.5042686602449344
INFO:moe.algorithm:Best expert Linear_Matern52 accuracy=1.021492694420564
INFO:ego.egor:Iteration 1/10: Best fun(x)=[-5.457730604665558, -0.06781945979561366, -0.04320859447174996] at x=[2.3314269035906396, 3.1263037010749186]
INFO:moe.algorithm:Best expert Linear_Matern52 accuracy=0.00000000000000017763568394002506
INFO:moe.algorithm:Best expert Linear_Matern52 accuracy=0.1939112812699915
INFO:moe.algorithm:Best expert Linear_Matern52 accuracy=0.517482675146703
INFO:ego.egor:Iteration 2/10: Best fun(x)=[-5.501536007846931, -0.019149157610847567, -0.0013644668747758715] at x=[2.3308993267120792, 3.1706366811348516]
INFO:moe.algorithm:Best expert Linear_SquaredExponential accuracy=0.0000000000000002220446049250313
INFO:moe.algorithm:Best expert Quadratic_SquaredExponential accuracy=0.07589117426202056
I

Optimization f=[-5.505490783000099, -0.008697078904546185, -3.0447453745807707e-05] at [2.330193092364205, 3.1752976906358943]


In [16]:
help(egor)

Help on Optimizer in module builtins object:

class Optimizer(object)
 |  Optimizer(xlimits, n_start=20, n_doe=10, regression_spec=7, correlation_spec=15, infill_strategy=1, infill_optimizer=1)
 |  
 |  Optimizer constructor
 |  
 |  Parameters
 |  
 |      xlimits (array[nx, 2]):
 |          Bounds of th nx components of the input x (eg. [[lower_1, upper_1], ..., [lower_nx, upper_nx]])
 |  
 |      n_start (int > 0):
 |          Number of runs of infill strategy optimizations (best result taken)
 |  
 |      n_doe (int > 0):
 |          Number of samples of initial LHS sampling (used when DOE not provided by the user)
 |  
 |      regr_spec (RegressionSpec flags, an int in [1, 7]):
 |          Specification of regression models used in gaussian processes.
 |          Can be RegressionSpec.CONSTANT (1), RegressionSpec.LINEAR (2), RegressionSpec.QUADRATIC (4) or
 |          any bit-wise union of these values (e.g. RegressionSpec.CONSTANT | RegressionSpec.LINEAR)
 |  
 |      corr_spec (